In [1]:
from chain_logging import langsmith
from dotenv import load_dotenv

load_dotenv()

langsmith("langchain_RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_RAG


### 대화 내용을 기억하는 기능을 추가

In [2]:
import os
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from vectorstore_manager import (
    load_vectorstore,
    create_vectorstore,
    create_conversaion_memory,
)
from config import PDF_PATH, VECTORSTORE_PATH, MODEL_PATH, LOCAL_MODEL_PATH, MODEL_NAME
from utils import get_embeddings, format_docs_with_print, create_prompt
from load_model import load_model, create_model

# Load or create vectorstore
if os.path.exists(VECTORSTORE_PATH):
    print("Loading existing vectorstore...")
    embeddings = get_embeddings()
    loaded_vectorstore = load_vectorstore(VECTORSTORE_PATH, embeddings)
else:
    print("Vectorstore not found. Creating a new one...")
    embeddings = get_embeddings()
    loaded_vectorstore = create_vectorstore(PDF_PATH, VECTORSTORE_PATH, embeddings)

# 대화를 기억할 메모리 객체 생성
memory = create_conversaion_memory("chat_history", "answer")

# 단계 5: 리트리버 생성
k = 3
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": k})

# 단계 6: 프롬프트 생성
prompt = create_prompt()

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
# llm = ChatOllama(model=MODEL_PATH, temperature=0)

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name=MODEL_PATH,  # 모델명
)

# if os.path.exists(LOCAL_MODEL_PATH):
#     print("Loading existing Local Model...")
#     llm = load_model(LOCAL_MODEL_PATH)
# else:
#     print("Local Model not found. Creating a new one...")
#     create_model(MODEL_NAME,LOCAL_MODEL_PATH)
#     llm = load_model(LOCAL_MODEL_PATH)


# 단계 8: 체인 생성
# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

Loading existing vectorstore...


c:\Users\SSAFY\ssafy\자율프로젝트\S11P31S102\AI\chain\vectorstore_manager.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [4]:
from chain_output_stream import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "연차는 어떻게 사용할 수 있나요?"
# question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갔다오는데 비용은 대충 어떻게 계산하죠?"
# question = "아까 물어본 질문을 다시 알려주세요"
# question = "급여 지급일은 언제인가요?"
question = "언제 퇴직금이 지급되나요?"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
answer = stream_response(response, return_output=True)

# 메모리에 대화 내용을 저장
memory.save_context({"question": question}, {"answer": answer})



[HUMAN]
언제 퇴직금이 지급되나요?


=== Retrieved Documents ===

Document 1:
..\kisa_pdf\3_09_급여지급규칙(240214).pdf |  page: 9
[별지제4호]<개정2009. 12. 24>퇴직금 중간정산 신청서결재담  당 접수담 당
신청일자신청번호부서(팀) 성명(사번)직   급 입사일자주   소정산기간   년 ~    년 위상기명본인은급여규정제17조및급여지급규칙제13조에따라퇴직금중간정산을신청합니다.
--------------------------------------------------

Document 2:
..\kisa_pdf\3_07_취업규칙(240214).pdf |  page: 4
1. 15일의연차유급휴가  2. 입사후1년미만의기간에대해발생하는연차유급휴가(단, 입사당해연도에발생한연차유급휴가일수는제외)  ⑦ 퇴직시에는 입사일을기준으로연차유급휴가를 정산한다.<신설2021.12.28.>제22조(공가) 직원이다음각호의어느하나에해당하는경우에는이에필요한기간동안공가를부여한다.  1. 병역법기타다른법령에의한징병검사･동원또는훈련에참가할때  2. 국회･법원･검찰기타국가기관의공무로인하여소환된때  3. 법률의규정에의하여투표에참가할때  4. 천재지변, 교통차단등기타의사유로출근이불가능할때  5. 기타원장이필요하다고인정할때  6. 「산업안전보건법」 제43조에따른건강진단또는「국민건강보험법」 제52조에따른건강검진을받을때<신설2019. 3.28>제23조(병가) ①직원이다음각호의어느하나에해당하는경우에는연60일의범위내에서병가를부여할수있다. 단, 공무상질병또는상해로인한경우에는그기간을연180일범위내에서부여할수있다. <개정2014. 8. 28., 2022. 5. 27.>  1. 질병또는부상으로인하여직무를수행할수없을때
--------------------------------------------------

Document 3:
..\kisa_pdf\2_06_급여규정(240221).pdf |  page: 1
여는신규채용,승진,정직,감봉또